# 5장 판다스 3편

## 주요 내용

`Series`와 `DataFrame` 객체를로부터 기초 통계 자료를 추출하는 방식을 다룬다.

* 합, 누적합
* 상관관계, 공분산
* 중복값 처리

## 기본 설정

`pandas` 라이브러리는 보통 `pd` 라는 별칭으로 사용된다.

In [1]:
import pandas as pd
import numpy as np

랜덤 시드, 어레이 내부에 사용되는 부동소수점 정확도, 도표 크기 지정 옵션 등은 이전과 동일하다.

In [2]:
np.random.seed(12345)
np.set_printoptions(precision=4, suppress=True)

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

`Series`와 `DataFrame`을 표로 보여줄 때 사용되는 행의 수를 20으로 지정한다. 
기본 값은 60이다.

In [3]:
PREVIOUS_MAX_ROWS = pd.options.display.max_rows # 원래 60이 기본.
pd.options.display.max_rows = 20

## 5.3 기초 통계 함수

In [ ]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5],
                   [np.nan, np.nan], [0.75, -1.3]],
                  index=['a', 'b', 'c', 'd'],
                  columns=['one', 'two'])
df

In [ ]:
df.sum()

In [ ]:
df.sum(axis='columns')

In [ ]:
df.mean(axis='columns', skipna=False)

In [ ]:
df.idxmax()

In [ ]:
df.cumsum()

In [ ]:
df.describe()

In [ ]:
obj = pd.Series(['a', 'a', 'b', 'c'] * 4)
obj.describe()

### 5.3.1 상관관계와 공분산

conda install pandas-datareader

In [ ]:
price = pd.read_pickle('examples/yahoo_price.pkl')
volume = pd.read_pickle('examples/yahoo_volume.pkl')

import pandas_datareader.data as web
all_data = {ticker: web.get_data_yahoo(ticker)
            for ticker in ['AAPL', 'IBM', 'MSFT', 'GOOG']}

price = pd.DataFrame({ticker: data['Adj Close']
                     for ticker, data in all_data.items()})
volume = pd.DataFrame({ticker: data['Volume']
                      for ticker, data in all_data.items()})

In [ ]:
returns = price.pct_change()
returns.tail()

In [ ]:
returns['MSFT'].corr(returns['IBM'])
returns['MSFT'].cov(returns['IBM'])

In [ ]:
returns.MSFT.corr(returns.IBM)

In [ ]:
returns.corr()
returns.cov()

In [ ]:
returns.corrwith(returns.IBM)

In [ ]:
returns.corrwith(volume)

### 5.3.2 중복값 처리

In [ ]:
obj = pd.Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])

In [ ]:
uniques = obj.unique()
uniques

In [ ]:
obj.value_counts()

In [ ]:
pd.value_counts(obj.values, sort=False)

In [ ]:
obj
mask = obj.isin(['b', 'c'])
mask
obj[mask]

In [ ]:
to_match = pd.Series(['c', 'a', 'b', 'b', 'c', 'a'])
unique_vals = pd.Series(['c', 'b', 'a'])
pd.Index(unique_vals).get_indexer(to_match)

In [ ]:
data = pd.DataFrame({'Qu1': [1, 3, 4, 3, 4],
                     'Qu2': [2, 3, 1, 2, 3],
                     'Qu3': [1, 5, 2, 4, 4]})
data

In [ ]:
result = data.apply(pd.value_counts).fillna(0)
result